In [1]:
# Import the Client Class
from mlflow.tracking import MlflowClient

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db" 

# Instantiate the Client object
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
# Check best runs in experiment id 1 (all models)
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids=1,
    filter_string='metrics.precision > 0.523',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=7,
    order_by=['metrics.precision DESC']
)

for run in runs:
    print(f"run id: {run.info.run_id},\
        precision: {run.data.metrics['precision']:.4f},\
        recall: {run.data.metrics['recall']:.4f}")

run id: 3f9cccee87e64d07b1f3ebd8ae23d099,        precision: 0.5373,        recall: 0.7500
run id: 2cd83bd14e6a4dd7a2ad2539c20b732a,        precision: 0.5333,        recall: 0.5000


In [4]:
# Promoting models to model registry
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [5]:
# Register the default Random Forest Classifier
register_name = 'marketing-campaign-classifier'
rfc_log_model = 'runs:/3f9cccee87e64d07b1f3ebd8ae23d099/model'

client.create_model_version(
    name=register_name,
    source="mlruns/1/3f9cccee87e64d07b1f3ebd8ae23d099/artifacts/model", 
    run_id=rfc_log_model,
    description=f"Random Forest Classifier with default parameter\
        showed to be the best model",
)

# Transition it to staging
client.transition_model_version_stage(
    name=register_name,
    version=3,
    stage="staging",
    archive_existing_versions=False
)

2023/06/22 10:33:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: marketing-campaign-classifier, version 3


<ModelVersion: aliases=[], creation_timestamp=1687419228349, current_stage='Staging', description=('Random Forest Classifier with default parameter        showed to be the best '
 'model'), last_updated_timestamp=1687419228439, name='marketing-campaign-classifier', run_id='runs:/3f9cccee87e64d07b1f3ebd8ae23d099/model', run_link=None, source='mlruns/1/3f9cccee87e64d07b1f3ebd8ae23d099/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [6]:
# Register the GradientBoosting Classifier (best model)
gbc_log_model = 'runs:/dea7e33c2f064ec28b844a20a4dbf41c/model'
client.create_model_version(
    name=register_name,
    source="mlruns/8/dea7e33c2f064ec28b844a20a4dbf41c/artifacts/model", 
    run_id=gbc_log_model,
    description=f"Gradient Boosting Classifier turned out to be the 'best' model",
)
# Transition it to production
client.transition_model_version_stage(
    name=register_name,
    version=4,
    stage="production",
    archive_existing_versions=False
)

2023/06/22 10:33:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: marketing-campaign-classifier, version 4


<ModelVersion: aliases=[], creation_timestamp=1687419228597, current_stage='Production', description="Gradient Boosting Classifier turned out to be the 'best' model", last_updated_timestamp=1687419228680, name='marketing-campaign-classifier', run_id='runs:/dea7e33c2f064ec28b844a20a4dbf41c/model', run_link=None, source='mlruns/8/dea7e33c2f064ec28b844a20a4dbf41c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [7]:
# Check the list of latest model versions
latest_versions = client.get_latest_versions(name=register_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: None
version: 3, stage: Staging
version: 4, stage: Production


In [8]:
# See which model to promote 